In [2]:
from pathlib import Path


try:
    import google.colab
    IN_COLAB = True
except ImportError:
    IN_COLAB = False

if IN_COLAB and Path.cwd().name == "content":
    if not Path("quantum-jam-chirimbolo").exists():
        !git clone https://github.com/segusantos/quantum-jam-chirimbolo.git
    %cd quantum-jam-chirimbolo
    !pip install .

In [2]:
import numpy as np
from qiskit import QuantumCircuit
from qiskit.quantum_info import Pauli, SparsePauliOp, Statevector, Operator
from qiskit.visualization import plot_histogram, plot_bloch_multivector, plot_bloch_vector
from qiskit_aer import AerSimulator
from qiskit.circuit import Parameter, ParameterVector
import qiskit.qasm3
from qiskit_ibm_runtime.fake_provider import FakeVigoV2
from qiskit.transpiler.preset_passmanagers import generate_preset_pass_manager
from qiskit_ibm_runtime import SamplerV2 as Sampler, EstimatorV2 as Estimator, QiskitRuntimeService

In [3]:
from src.test import test

test()

This is a test!
